# Datenvorverarbereitung/Stichproben und Vor-Analyse

Zelle 1 - Setup und Imports:

Importiert alle benötigten Bibliotheken
Setzt grundlegende Darstellungsoptionen
Konfiguriert den Plot-Stil

In [1]:
#Setup und Imports
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import os

# Setzen der Display-Optionen für bessere Lesbarkeit
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', 1000)

# Warnung unterdrücken
import warnings
warnings.filterwarnings('ignore')

# Plotstil festlegen (verwenden einen basic matplotlib style)
plt.style.use('default')
# Seaborn Plotting Style setzen
sns.set_theme(style="whitegrid")


In [ ]:
# ignorieren. Nur zum Testen des Verzechnisses
import os
print("Aktuelles Arbeitsverzeichnis:", os.getcwd())
print("\nDateien im aktuellen Verzeichnis:")
print(os.listdir())

Daten laden und erste Inspektion:

Lädt den Datensatz
Zeigt grundlegende Informationen wie Dimensionen und Datentypen
Überprüft auf fehlende Werte

In [ ]:
# Zelle 2: Daten laden und erste Inspektion
# Daten einlesen mit zusätzlichen Parametern für robusteres Parsing
# df = pd.read_csv('https://raw.githubusercontent.com/Takual/project/refs/heads/main/data/raw/videos_ab_60s.csv', 
#                 encoding='utf-8',
#                 sep=';',  # Semikolon als Trennzeichen
#                 decimal=',',  # Komma als Dezimaltrennzeichen
#                 thousands='.',  # Punkt als Tausendertrennzeichen
#                on_bad_lines='warn')  # Warnung bei problematischen Zeilen

df = pd.read_csv('../data/raw/videos_ohnelive_ohneshorts.csv', encoding='utf-8', sep=';', decimal=',', thousands='.')

# Überblick über den Datensatz
print("Dimensionen des Datensatzes:", df.shape)
print("\nInformationen über den Datensatz:")
print(df.info())

# Erste Zeilen anzeigen
print("\nErste Zeilen des Datensatzes:")
print(df.head())

# Überprüfung auf fehlende Werte
print("\nAnzahl fehlender Werte pro Spalte:")
print(df.isnull().sum())

#### Ignore 
(Hier noch ein Überblick über die Spalten und die Datenstruktur, auftgeteilt in Gruppen)

In [ ]:
import pandas as pd

def show_column_groups_table(df):
    def shorten_column_name(name, max_length=30):
        """Kürzt lange Spaltennamen sinnvoll"""
        if len(name) <= max_length:
            return name
        
        # Versuche, sinnvolle Abkürzungen zu erstellen
        words = name.split('_')
        if len(words) > 1:
            # Nehme Anfangsbuchstaben und letzte Wörter
            shortened = ''.join(word[0].upper() for word in words[:-1]) + '_' + words[-1]
            return shortened[:max_length]
        
        # Für einen langen Namen ohne Unterstrich
        return name[:15] + '...' + name[-10:]

    # Gruppiere Spalten nach Datentyp
    numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns.tolist()
    datetime_cols = df.select_dtypes(include=['datetime64']).columns.tolist()
    categorical_cols = df.select_dtypes(include=['object', 'category']).columns.tolist()
    boolean_cols = df.select_dtypes(include=['bool']).columns.tolist()
    
    # Kürze Spaltennamen
    numeric_cols_short = [shorten_column_name(col) for col in numeric_cols]
    datetime_cols_short = [shorten_column_name(col) for col in datetime_cols]
    categorical_cols_short = [shorten_column_name(col) for col in categorical_cols]
    boolean_cols_short = [shorten_column_name(col) for col in boolean_cols]
    
    # Erstelle einen DataFrame für die Übersicht
    column_groups = pd.DataFrame({
        'Datentyp': [
            'Numerische Spalten', 
            'Datetime Spalten', 
            'Kategorische/Text Spalten',
            'Boolesche Spalten'
        ],
        'Spalten': [
            ', '.join(numeric_cols_short) if numeric_cols_short else '-',
            ', '.join(datetime_cols_short) if datetime_cols_short else '-',
            ', '.join(categorical_cols_short) if categorical_cols_short else '-',
            ', '.join(boolean_cols_short) if boolean_cols_short else '-'
        ],
        'Anzahl': [
            len(numeric_cols),
            len(datetime_cols),
            len(categorical_cols),
            len(boolean_cols)
        ],
        'Original Namen': [
            ', '.join(numeric_cols) if numeric_cols else '-',
            ', '.join(datetime_cols) if datetime_cols else '-',
            ', '.join(categorical_cols) if categorical_cols else '-',
            ', '.join(boolean_cols) if boolean_cols else '-'
        ]
    })
    
    # Formatiere die Ausgabe
    pd.set_option('display.max_colwidth', None)
    print(column_groups)
    
    # Zusätzliche Gesamtinformationen
    print(f"\nGesamtanzahl Spalten: {len(df.columns)}")
    
    return column_groups

# Funktion aufrufen
column_overview = show_column_groups_table(df)

Datenbereinigung und Transformation:

Konvertiert Datums-Strings in datetime-Objekte


In [ ]:
# Zelle 3: Datenbereinigung
# Datumskonvertierung mit verbesserte Fehlerbehandlung
if 'publish_date' in df.columns:
    df['publish_date'] = pd.to_datetime(df['publish_date'], format='%d.%m.%Y %H:%M', errors='coerce')
    df['publish_hour'] = df['publish_date'].dt.hour
    df['publish_weekday'] = df['publish_date'].dt.dayofweek

print("Datentypen nach Bereinigung:")
print(df.dtypes)


#### Feature Engineering
Diese Zelle bereitet die Features für die Modellierung vor. Ein wichtiger Schritt ist die Umwandlung der kategorialen Variable "Thema" in Dummy-Variablen oder Indikatorvariable. Dabei wird jede Kategorie (wie "Politik", "Wirtschaft" etc.) in eine eigene Spalte umgewandelt, die nur 0 und 1 enthält. Dies ist notwendig, um Multikollinarität zu vermeiden. Außerdem werden hier die Ziel- und Prädiktorvariablen definiert, die für die späteren Modelle verwendet werden.

In [3]:
# Zelle 4: Feature Engineering
# Dummy-Variablen für Thema erstellen
if 'Thema' in df.columns:
    theme_dummies = pd.get_dummies(df['Thema'], prefix='theme')
    df = pd.concat([df, theme_dummies], axis=1)

# Zielvariablen definieren
target_columns = ['durchschnittliche_wiedergabedauer', 'aufrufe', 'Klickrate der Impressionen (%)']

# Features für die Modellierung auswählen
feature_columns = [col for col in df.columns if col not in target_columns]

#### Skalierung der Features
In dieser Zelle werden die numerischen Features standardisiert. Dies ist wichtig, weil die verschiedenen Features sehr unterschiedliche Größenordnungen haben (z.B. Video-Länge in Sekunden vs. Bewertungen von 1-10). Die Standardisierung sorgt dafür, dass alle numerischen Features einen Mittelwert von 0 und eine Standardabweichung von 1 haben. Dies verhindert, dass Features mit großen Zahlen das Modell unverhältnismäßig stark beeinflussen.

In [ ]:
# Zelle 5: Selektive Skalierung der Features
from sklearn.preprocessing import StandardScaler

# Scaler für bestimmte numerische Features
scaler = StandardScaler()

# Spalten, die skaliert werden sollten (große Wertebereiche)
columns_to_scale = [
    'video_length_seconds', 'wiedergabezeit_minuten', 'durchschnittliche_wiedergabedauer',
    'aufrufe', 'likes', 'dislikes', 'geteilte_inhalte', 'kommentare', 'Impressionen'
]

# Skalieren nur dieser Spalten
df[columns_to_scale] = scaler.fit_transform(df[columns_to_scale])

print("\nFeature-Columns:", feature_columns)
print("\nNumerische Features, die skaliert wurden:", columns_to_scale)


#### Train-Test-Split
Diese Zelle teilt die Daten in drei Teile auf: Trainings-, Validierungs- und Testdaten. Der Trainingsdatensatz (60%) wird verwendet, um das Modell zu trainieren. Der Validierungsdatensatz (20%) dient dazu, die Modellleistung während des Trainings zu überprüfen und  zu optimieren. Der Testdatensatz (20%) wird erst ganz am Ende verwendet, um die finale Modellperformance zu evaluieren.

In [ ]:
# Zelle 6: Train-Test-Split
# Features und Targets separieren
X = df[feature_columns]
y = df[target_columns]

# Erste Aufteilung in temporäres Training+Validierung und Test
X_temp, X_test, y_temp, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Zweite Aufteilung des temporären Sets in Training und Validierung
X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp, test_size=0.25, random_state=42  # 0.25 x 0.8 = 0.2
)

print("\nDimensionen der Datensätze:")
print(f"Training: {X_train.shape}, {y_train.shape}")
print(f"Validierung: {X_val.shape}, {y_val.shape}")
print(f"Test: {X_test.shape}, {y_test.shape}")

#### Daten speichern
In der letzten Zelle werden alle verarbeiteten Daten gespeichert, damit sie in späteren Analysen wieder verwendet werden können. Die Daten werden sowohl im Pickle-Format (für Python-spezifische Objekte wie den Scaler) als auch als CSV-Dateien gespeichert. CSVs sind dabei besonders nützlich für schnelle Inspektionen der Daten. Wichtig ist auch, dass der Scaler mitgespeichert wird, damit neue Daten auf die gleiche Weise transformiert werden können.
Dies stellt sicher, dass wir einen sauberen, reproduzierbaren Datensatz für unsere Analysen haben und dass alle Transformationen konsistent auf zukünftige Daten angewendet werden können.

In [6]:
# Zelle 7: Daten speichern
# Ordner für verarbeitete Daten erstellen
if not os.path.exists('data/processed'):
    os.makedirs('data/processed')

# Daten als Pickle-Datei speichern
with open('../data/processed/processed_data.pkl', 'wb') as f:
    pickle.dump({
        'X_train': X_train,
        'X_val': X_val,
        'X_test': X_test,
        'y_train': y_train,
        'y_val': y_val,
        'y_test': y_test,
        'scaler': scaler,
        'feature_columns': feature_columns,
        'target_columns': target_columns
    }, f)

print("\nDaten wurden erfolgreich gespeichert in 'data/processed/processed_data.pkl'")


Daten wurden erfolgreich gespeichert in 'data/processed/processed_data.pkl'


In [7]:
# Optional: Auch als CSV speichern für einfachere Inspektion
X_train.to_csv('../data/processed/X_train.csv', sep=';', decimal=',', index=False)
y_train.to_csv('../data/processed/y_train.csv', sep=';', decimal=',', index=False)
X_val.to_csv('../data/processed/X_val.csv', sep=';', decimal=',', index=False)
y_val.to_csv('../data/processed/y_val.csv', sep=';', decimal=',', index=False)
X_test.to_csv('../data/processed/X_test.csv', sep=';', decimal=',', index=False)
y_test.to_csv('../data/processed/y_test.csv', sep=';', decimal=',', index=False)
